In [160]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage,ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os
import duckduckgo_search as DDGS
print("✅ All imports successful")

✅ All imports successful


In [161]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("openai_key")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded")

✅ API key loaded


In [192]:
@tool
def get_weather(city: str) -> str:
    """Returns simulated weather for a given city."""
    fake_weather = {
        "lagos": "Sunny, 30°C",
        "london": "Cloudy, 18°C",
        "new york": "Rainy, 22°C"
    }
    return fake_weather.get(city.lower(), "Weather data not available.")


@tool
def define_word(word: str) -> str:
    """Returns the definition of a word."""
    dictionary = {
        "ephemeral": "Lasting for a very short time.",
        "ubiquitous": "Present, appearing, or found everywhere.",
        "resilient": "Able to recover quickly from difficulties."
    }
    return dictionary.get(word.lower(), "Definition not found.")



from duckduckgo_search import DDGS

@tool
def web_search(query: str) -> str:
    """
    DuckDuckGo web search.
    """
    ddgs=DDGS()
    results = ddgs.text(
        query,
        max_results=3
    )

    if not results:
        return "No results found."

    return "\n\n".join(
        f"{r.get('title','')}\n{r.get('body','')}"
        for r in results
    )


In [193]:
from duckduckgo_search import DDGS
print(DDGS)


<class 'duckduckgo_search.duckduckgo_search.DDGS'>


In [194]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # Lower temperature for more precise tool usage
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [195]:

tools = [get_weather, define_word, web_search]

llm_with_tools = llm.bind_tools(tools)

In [196]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """
    Decide next step based on last message.
    
    If LLM called a tool → go to 'tools' node
    If LLM provided final answer → go to END
    """
    last_message = state["messages"][-1]
    
    # Check if LLM made tool calls
    if last_message.tool_calls:
        return "tools"
    
    # No tool calls - we're done
    return "__end__"

print("✅ Conditional routing function defined")

✅ Conditional routing function defined


In [197]:
#system prompt that defines assistant behavior 

sys_msg= SystemMessage(
    content="""You are a friendly assistant with access to helpful tools.

    When the user asks about weather:
- Extract the city name
- Call the get_weather tool with argument `city`
    when asked questions related to word definitions use the define_word tool and provide answers from the hard coded answers in  the tool.
    when asked general questions use the web_search tool
    Always provide concise and accurate responses.
    Only use tools when necessary - for simple questions, answer directly.""")
def assistant(state:MessagesState)-> dict:
    """
    The assistant node - processes messages and generates response.
    """
    messages = [sys_msg] + state["messages"]
    response = llm_with_tools.invoke(messages)

    print("DEBUG MESSAGE:", response)
    return {"messages": [response]}


In [198]:
builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START,"assistant")

builder.add_conditional_edges(
    "assistant",
    should_continue,
    {
        "tools": "tools",
        "__end__": END
    }
)

builder.add_edge("tools", "assistant")


print("[OK] LangGraph agent ready!")


[OK] LangGraph agent ready!


In [199]:
#create a memory checkpointer (stores in memory)
memory=MemorySaver()

#compile the graph with memory
agent=builder.compile(checkpointer=memory)

In [200]:
# def run_agent(query: str,thread_id: str = "test_session"):
#     result = agent.invoke(
#         {"messages": [HumanMessage(content=query)]}, config={"configurable": {"thread_id": thread_id}}
#     )
#     return result["messages"][-1].content

In [201]:
# # First query
# run_agent("what is the weather in lagos")

In [202]:



# Helper function
def run_agent(user_input: str, thread_id: str = "test_session"):
    """
    Run the agent and display the conversation.
    """
    print(f"\n{'='*70}")
    print(f"👤 User: {user_input}")
    print(f"{'='*70}\n")
    
    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    for message in result["messages"]:
        if isinstance(message, HumanMessage):
            continue  # Already printed
        elif isinstance(message, AIMessage):
            if message.tool_calls:
                print(f"🤖 Agent: [Calling tool: {message.tool_calls[0]['name']}]")
            else:
                print(f"🤖 Agent: {message.content}")
        elif isinstance(message, ToolMessage):
            print(f"🔧 Tool Result: {message.content[:100]}..." if len(message.content) > 100 else f"🔧 Tool Result: {message.content}")
    
    print(f"\n{'='*70}\n")

print("✅ Test function ready")

✅ Test function ready


In [203]:
# First query
run_agent("what is the weather in lagos")


👤 User: what is the weather in lagos

DEBUG MESSAGE: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 188, 'total_tokens': 203, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CuTbJJ8MIGq8FsZvrxINtR900aPst', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b8bc0-3817-7802-86bf-59d1b56a2e54-0' tool_calls=[{'name': 'get_weather', 'args': {'city': 'lagos'}, 'id': 'call_KOJYG2qKJwfSyTzVDcOxUva0', 'type': 'tool_call'}] usage_metadata={'input_tokens': 188, 'output_tokens': 15, 'total_tokens': 203, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 

In [204]:
# First query
run_agent("what does resilient mean")


👤 User: what does resilient mean

DEBUG MESSAGE: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 242, 'total_tokens': 258, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CuTbLmaCaljbo3AsW7TYjN4Vpl1Pv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b8bc0-4182-7e41-8c39-db7149b9cbc2-0' tool_calls=[{'name': 'define_word', 'args': {'word': 'resilient'}, 'id': 'call_wqmKLb1O0QWJJ22oUz9gaVO4', 'type': 'tool_call'}] usage_metadata={'input_tokens': 242, 'output_tokens': 16, 'total_tokens': 258, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 

In [205]:
# First query
run_agent("who is the current president of iraq")


👤 User: who is the current president of iraq

DEBUG MESSAGE: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 302, 'total_tokens': 322, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CuTbO42ZHbVeUnGcAFR7fQAbUZyiV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b8bc0-4a06-7141-be21-d4007b3c3401-0' tool_calls=[{'name': 'web_search', 'args': {'query': 'current president of Iraq 2023'}, 'id': 'call_H0k9r4Cb4n2b8ku8O2pxPfA9', 'type': 'tool_call'}] usage_metadata={'input_tokens': 302, 'output_tokens': 20, 'total_tokens': 322, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_det

C:\Users\User\AppData\Local\Temp\ipykernel_15868\1561609777.py:31: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs=DDGS()


DEBUG MESSAGE: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 532, 'total_tokens': 549, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CuTbR1lvtwXYI2vROfgpqhRpeAo5I', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b8bc0-5a1b-7cc2-9abd-6d22d173cc3f-0' tool_calls=[{'name': 'web_search', 'args': {'query': 'current president of Iraq'}, 'id': 'call_vdriWIjlUQpd7wr8Zud3dYIK', 'type': 'tool_call'}] usage_metadata={'input_tokens': 532, 'output_tokens': 17, 'total_tokens': 549, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


C:\Users\User\AppData\Local\Temp\ipykernel_15868\1561609777.py:31: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs=DDGS()


DEBUG MESSAGE: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 779, 'total_tokens': 798, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CuTbVzAHU2MQ7tEfBGBUTBUMfhTnk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b8bc0-689c-7cf2-b06f-5476e5966284-0' tool_calls=[{'name': 'web_search', 'args': {'query': 'Iraq president 2023'}, 'id': 'call_NcSPN4axFoSShGXItn8U9IRw', 'type': 'tool_call'}] usage_metadata={'input_tokens': 779, 'output_tokens': 19, 'total_tokens': 798, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


C:\Users\User\AppData\Local\Temp\ipykernel_15868\1561609777.py:31: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs=DDGS()


DEBUG MESSAGE: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 992, 'total_tokens': 1012, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CuTbZwgGSNReiGO4WpAwYgudmtwxm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b8bc0-77a6-7f83-8088-387af8474c2d-0' tool_calls=[{'name': 'web_search', 'args': {'query': 'Iraq president 2023 name'}, 'id': 'call_MdiAnlQypcLDMbgJtfhySDtH', 'type': 'tool_call'}] usage_metadata={'input_tokens': 992, 'output_tokens': 20, 'total_tokens': 1012, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


C:\Users\User\AppData\Local\Temp\ipykernel_15868\1561609777.py:31: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs=DDGS()


DEBUG MESSAGE: content="I couldn't find the current president of Iraq. However, as of my last update, the president was Abdul Latif Rashid, who took office in October 2022. Please verify with a reliable source for the most current information." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 1024, 'total_tokens': 1072, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CuTbcACk71Nqcd7cT6eRRdxqpOcMj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b8bc0-858b-7770-ab17-ee08f6982c06-0' usage_metadata={'input_tokens': 1024, 'output_tokens': 48, 'total_tokens': 1072, 'input_token_details': {'audio': 0, 'cache_read': 